Training, Validation and testing dataset were labeled by batch manually

In [1]:
import sys
sys.path.append("../../")
import os
import numpy as np
import pandas as pd
import HeadCT_motion_correction_PAR.functions_collection as ff

In [5]:
# data_path = '/Volumes/IRB2020P002624-DATA/zhennongchen/motion_correction/data'
data_path = '/mnt/camca_NAS/motion_correction/data'

In [6]:
l = ff.find_all_target_files(['*/*'],os.path.join(data_path,'raw_data/nii-images/thin_slice'))
print(len(l))

112


In [7]:
file = os.path.join(data_path,'Patient_list/Patient_list_batch_PAR.xlsx')
data = pd.read_excel(file)
print(data.shape[0])

100


# spreadsheet for all slices - slice thickness = 2.5mm

In [8]:
Result = []
random_list = np.arange(1,101,1)
slice_list = [[5,20], [20,35], [35,50]]

for i in range(0,data.shape[0]):
    case = data.iloc[i]
    patient_id = case['PatientID']
    patient_subid = case['AccessionNumber']
    batch = case['batch']
    
    if batch == 5:
        train = 'test'
    else:
        train = 'train'
       
        
    # motion-free
    motion_free_file = os.path.join(data_path,'simulated_data_3D_spline',patient_id,patient_subid,'static/image_data/recon_partial.nii.gz')
    assert os.path.isfile(motion_free_file) == 1
    # motion-free downsampled and cropped
    motion_free_ds_file = ''
    # motion_free_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,'static/image_data/recon_partial_ds.nii.gz')
    # assert os.path.isfile(motion_free_ds_file) == 1

    if case['ConfirmedMotion'] == False:

        for j in random_list:
            motion_name = 'random_'+ str(j)

            # motion image
            motion_corrupted_file = os.path.join(data_path,'simulated_data_3D_spline_6degrees',patient_id,patient_subid,motion_name,'image_data/recon_partial.nii.gz')
            # assert os.path.isfile(motion_corrupted_file) == 1

            # motion image downsampled and cropped
            motion_corrupted_ds_file = ''
            # motion_corrupted_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,motion_name,'image_data/recon_partial_ds.nii.gz')

            # motion parameters
            motion_parameter_file = os.path.join(data_path,'simulated_data_3D_spline_6degrees',patient_id,patient_subid,motion_name,'motion_parameters_7CP.npy')
            
            # PAR
            PAR_folders = os.path.join(data_path,'PAR_3D_spline_6degrees',patient_id,patient_subid, motion_name, 'slice_0_to_50')
                
            # PAR file
            PAR_file = os.path.join(PAR_folders, 'PARs_ds_crop.nii.gz')
            # assert os.path.isfile(PAR_file) == 1
            
            # apply true MVF to PAR
            image_from_PAR_MVF = ''

            for s in slice_list:
                start_slice = s[0]
                end_slice = s[1]
            
                Result.append([train, batch, patient_id, patient_subid, motion_name, start_slice, end_slice, motion_free_file, motion_free_ds_file, motion_corrupted_file, 
                                motion_corrupted_ds_file, motion_parameter_file, PAR_file, image_from_PAR_MVF])

    else: # True
        motion_name = 'real_data'
        continue

column_list = ['group','batch','PatientID','AccessionNumber','MotionName','PAR_start_slice', 'PAR_end_slice', 'MotionFree_File', 'MotionFreeDs_File',
                'Motion_File', 'MotionDs_File', 'MotionParam','PAR_File','ImageFromPARMVF']
df = pd.DataFrame(Result, columns = column_list)
df.to_excel(os.path.join(data_path,'Patient_list/Patient_list_train_test_3D_spline_6degrees_PAR_downsampled_slice0-50_ablation_study_7CP.xlsx'),index=False)

# spreadsheet for particular slices, slice thickness = 2.5mm

In [5]:
Result = []
random_list = np.arange(1,101,1)

for i in range(0,data.shape[0]):
    case = data.iloc[i]
    patient_id = case['PatientID']
    patient_subid = case['AccessionNumber']
    batch = case['batch']
    
    if batch == 5:
        train = 'test'
    else:
        train = 'train'
       
        
    # motion-free
    motion_free_file = os.path.join(data_path,'simulated_data_3D_spline',patient_id,patient_subid,'static/image_data/recon_partial.nii.gz')
    assert os.path.isfile(motion_free_file) == 1
    # motion-free downsampled and cropped
    motion_free_ds_file = ''
    # motion_free_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,'static/image_data/recon_partial_ds.nii.gz')
    # assert os.path.isfile(motion_free_ds_file) == 1

    if case['ConfirmedMotion'] == False:

        for j in random_list:
            motion_name = 'random_'+ str(j)

            # motion image
            motion_corrupted_file = os.path.join(data_path,'simulated_data_3D_spline',patient_id,patient_subid,motion_name,'image_data/recon_partial.nii.gz')
            # assert os.path.isfile(motion_corrupted_file) == 1

            # motion image downsampled and cropped
            motion_corrupted_ds_file = ''
            # motion_corrupted_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,motion_name,'image_data/recon_partial_ds.nii.gz')

            # motion parameters
            motion_parameter_file = os.path.join(data_path,'simulated_data_3D_spline',patient_id,patient_subid,motion_name,'motion_parameters.npy')
            
            # PAR
            PAR_folders = os.path.join(data_path,'PAR_3D_spline',patient_id,patient_subid, motion_name, 'slice_20_to_35')

            end_slice = ff.find_timeframe(PAR_folders, 0 ,'_', '/')
            start_slice = end_slice - 15
                
            # PAR file
            PAR_file = os.path.join(PAR_folders,'ds', 'PARs_ds_crop.nii.gz')
            # assert os.path.isfile(PAR_file) == 1
            
            # apply true MVF to PAR
            image_from_PAR_MVF = ''
            # image_from_PAR_MVF = os.path.join(PAR_folders,'image_by_PAR_true_MVF', 'image_by_PAR_ds.nii.gz')
            # assert os.path.isfile(image_from_PAR_MVF) == 1
            
            Result.append([train, batch, patient_id, patient_subid, motion_name, start_slice, end_slice, motion_free_file, motion_free_ds_file, motion_corrupted_file, 
                            motion_corrupted_ds_file, motion_parameter_file, PAR_file, image_from_PAR_MVF])

    else: # True
        motion_name = 'real_data'
        continue
        # # motion image
        # motion_corrupted_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,'static/image_data/recon_partial.nii.gz')
        # # motion image downsampled and cropped
        # motion_corrupted_ds_file = ''#os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,'static/image_data/recon_partial_ds.nii.gz')
        
        # # motion parameters
        # motion_parameter_file = ''

        # # PAR
        # PAR_folders = os.path.join(data_path,'PAR_2D_spline',patient_id,patient_subid, motion_name, 'slice_5_to_20')

        # end_slice = ff.find_timeframe(PAR_folders, 0 ,'_', '/')
        # start_slice = end_slice - 15
                
        # # PAR file
        # PAR_file = os.path.join(PAR_folders,'ds', 'PARs_ds_crop.nii.gz')
        # # apply true MVF to PAR
        # image_from_PAR_MVF = ''#os.path.join(PAR_folders,'image_by_PAR_true_MVF', 'image_by_PAR_ds.nii.gz')


        # Result.append([train, batch, patient_id, patient_subid, motion_name, start_slice, end_slice, motion_free_file, motion_free_ds_file, motion_corrupted_file, 
        #                     motion_corrupted_ds_file, motion_parameter_file, PAR_file, image_from_PAR_MVF])
    

    
column_list = ['group','batch','PatientID','AccessionNumber','MotionName','PAR_start_slice', 'PAR_end_slice', 'MotionFree_File', 'MotionFreeDs_File',
                'Motion_File', 'MotionDs_File', 'MotionParam','PAR_File','ImageFromPARMVF']
df = pd.DataFrame(Result, columns = column_list)
df.to_excel(os.path.join(data_path,'Patient_list/Patient_list_train_test_3D_spline_PAR_downsampled_slice20-35.xlsx'),index=False)

# Spreadsheet for all slices, slice thickness = 0.625mm

In [7]:
Result = []
random_list = np.arange(1,51,1)
slice_list = [[60,75], [120,135], [180,195]]

for i in range(0,data.shape[0]):
    case = data.iloc[i]
    patient_id = case['PatientID']
    patient_subid = case['AccessionNumber']
    batch = case['batch']
    
    if batch == 5:
        train = 'test'
    else:
        train = 'train'
       
    # motion-free
    motion_free_file = os.path.join(data_path,'raw_data/nii-images/thin_slice',patient_id,patient_subid,'img-nii-0.625/img.nii.gz')
    assert os.path.isfile(motion_free_file) == 1
    # motion-free downsampled and cropped
    motion_free_ds_file = ''
    # motion_free_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,'static/image_data/recon_partial_ds.nii.gz')
    # assert os.path.isfile(motion_free_ds_file) == 1

    if case['ConfirmedMotion'] == False:

        for j in random_list:
            motion_name = 'random_'+ str(j)

            # motion image
            motion_corrupted_file = os.path.join(data_path,'simulated_data_3D_spline_6degrees_HR',patient_id,patient_subid,motion_name,'image_data/recon.nii.gz')
            # assert os.path.isfile(motion_corrupted_file) == 1

            # motion image downsampled and cropped
            motion_corrupted_ds_file = ''
            # motion_corrupted_ds_file = os.path.join(data_path,'simulated_data_2D_spline',patient_id,patient_subid,motion_name,'image_data/recon_partial_ds.nii.gz')

            # motion parameters
            motion_parameter_file = os.path.join(data_path,'simulated_data_3D_spline_6degrees_HR',patient_id,patient_subid,motion_name,'motion_parameters.npy')
            
            # PAR
            PAR_folders = os.path.join(data_path,'PAR_3D_spline_6degrees_HR',patient_id,patient_subid, motion_name, 'all_slices')
                
            # PAR file
            PAR_file = os.path.join(PAR_folders, 'PARs_ds_crop_anneal.nii.gz')
            # assert os.path.isfile(PAR_file) == 1
            
            # apply true MVF to PAR
            image_from_PAR_MVF = ''

            for s in slice_list:
                start_slice = s[0]
                end_slice = s[1]
            
                Result.append([train, batch, patient_id, patient_subid, motion_name, start_slice, end_slice, motion_free_file, motion_free_ds_file, motion_corrupted_file, 
                                motion_corrupted_ds_file, motion_parameter_file, PAR_file, image_from_PAR_MVF])

    else: # True
        motion_name = 'real_data'
        continue

column_list = ['group','batch','PatientID','AccessionNumber','MotionName','PAR_start_slice', 'PAR_end_slice', 'MotionFree_File', 'MotionFreeDs_File',
                'Motion_File', 'MotionDs_File', 'MotionParam','PAR_File','ImageFromPARMVF']
df = pd.DataFrame(Result, columns = column_list)
df.to_excel(os.path.join(data_path,'Patient_list/Patient_list_train_test_3D_spline_6degrees_PAR_downsampled_thinslicethickness.xlsx'),index=False)